In [1]:
import pandas as pd
import numpy as np


In [2]:
log_data = pd.read_csv('C:/Users/hugou/Downloads/challenge-2023/challenge-2023/données/log_fw_3.csv/log_fw_3.csv', sep=';', header=None)

In [3]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  object 
 1   1       999998 non-null   object 
 2   2       999998 non-null   object 
 3   3       999998 non-null   object 
 4   4       999998 non-null   float64
 5   5       999998 non-null   float64
 6   6       999998 non-null   float64
 7   7       999998 non-null   object 
 8   8       999998 non-null   object 
 9   9       0 non-null        float64
 10  10      999998 non-null   float64
dtypes: float64(5), object(6)
memory usage: 83.9+ MB


In [4]:
def to_datetime(x):
    return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f')

def prepare_log_data(log_data):
    log_data.columns = ['timestamp', 'IPsrc', 'IPdst', 'proto', 'portsrc', 'portdst', 'rule','action','7', '8','9']
    log_data.timestamp = to_datetime(log_data.timestamp)
    log_data.portdst = log_data.portdst.astype('object')
    log_data['port_type'] = log_data['portdst'].apply(lambda x: 'wk' if x < 1024 else ('reg' if x >= 1024 and x <= 49151 else 'priv'))
    return log_data

In [5]:
prepare_log_data(log_data)
log_data.head()

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,8,9,port_type
0,2023-02-12 03:59:03,66.249.69.180,17.17.17.17,TCP,57630.0,443.0,1.0,PERMIT,eth0,NaN,6.0,wk
1,2023-02-12 03:59:04,66.249.69.178,17.17.17.17,TCP,63808.0,443.0,1.0,PERMIT,eth0,NaN,6.0,wk
2,2023-02-12 03:59:08,221.11.125.141,17.17.17.17,TCP,43123.0,2376.0,999.0,DENY,eth0,NaN,6.0,reg
3,2023-02-12 03:59:18,183.136.225.42,17.17.17.17,TCP,8088.0,70.0,999.0,DENY,eth0,NaN,6.0,wk
4,2023-02-12 03:59:20,38.32.112.34,17.17.17.17,TCP,6356.0,443.0,1.0,PERMIT,eth0,NaN,6.0,wk


In [5]:
log_data[3].value_counts()



TCP    999998
Name: 3, dtype: int64

In [65]:
def to_datetime(x):
    return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S.%f')

def prepare_log_data(log_data):
    log_data.columns = ['timestamp', 'IPsrc', 'IPdst', 'proto', 'portsrc', 'portdst', 'rule','action','7', '8','9']
    log_data.timestamp = to_datetime(log_data.timestamp)
    log_data.portdst = log_data.portdst.astype('object')
    return log_data

In [8]:
data = log_data.copy()

In [9]:
data.pop('8')

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
999995   NaN
999996   NaN
999997   NaN
999998   NaN
999999   NaN
Name: 8, Length: 1000000, dtype: float64

In [10]:
data.head(10)

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9,port_type
0,2023-02-12 03:59:03,66.249.69.180,17.17.17.17,TCP,57630.0,443.0,1.0,PERMIT,eth0,6.0,wk
1,2023-02-12 03:59:04,66.249.69.178,17.17.17.17,TCP,63808.0,443.0,1.0,PERMIT,eth0,6.0,wk
2,2023-02-12 03:59:08,221.11.125.141,17.17.17.17,TCP,43123.0,2376.0,999.0,DENY,eth0,6.0,reg
3,2023-02-12 03:59:18,183.136.225.42,17.17.17.17,TCP,8088.0,70.0,999.0,DENY,eth0,6.0,wk
4,2023-02-12 03:59:20,38.32.112.34,17.17.17.17,TCP,6356.0,443.0,1.0,PERMIT,eth0,6.0,wk
5,2023-02-12 03:59:22,45.125.66.18,17.17.17.17,TCP,58438.0,23.0,7.0,DENY,eth0,6.0,wk
6,2023-02-12 03:59:29,107.170.242.5,17.17.17.17,TCP,39974.0,5093.0,999.0,DENY,eth0,6.0,reg
7,2023-02-12 03:59:33,45.93.16.171,17.17.17.17,TCP,42021.0,50802.0,999.0,DENY,eth0,6.0,priv
8,2023-02-12 03:59:35,157.55.39.59,17.17.17.17,TCP,45120.0,443.0,1.0,PERMIT,eth0,6.0,wk
9,2023-02-12 03:59:39,104.156.155.11,17.17.17.17,TCP,58728.0,9109.0,999.0,DENY,eth0,6.0,reg


In [11]:
data.portsrc = data.portsrc.astype('object')

In [12]:
data.dtypes

timestamp    datetime64[ns]
IPsrc                object
IPdst                object
proto                object
portsrc              object
portdst              object
rule                float64
action               object
7                    object
9                   float64
port_type            object
dtype: object

In [13]:
data.dropna(inplace=True)

In [14]:
data.head()

,timestamp,IPsrc,IPdst,proto,portsrc,portdst,rule,action,7,9,port_type
0,2023-02-12 03:59:03,66.249.69.180,17.17.17.17,TCP,57630.0,443.0,1.0,PERMIT,eth0,6.0,wk
1,2023-02-12 03:59:04,66.249.69.178,17.17.17.17,TCP,63808.0,443.0,1.0,PERMIT,eth0,6.0,wk
2,2023-02-12 03:59:08,221.11.125.141,17.17.17.17,TCP,43123.0,2376.0,999.0,DENY,eth0,6.0,reg
3,2023-02-12 03:59:18,183.136.225.42,17.17.17.17,TCP,8088.0,70.0,999.0,DENY,eth0,6.0,wk
4,2023-02-12 03:59:20,38.32.112.34,17.17.17.17,TCP,6356.0,443.0,1.0,PERMIT,eth0,6.0,wk


In [15]:
# get IPsrc, IPdst, portsrc, portdst, action, rule, 9 into a new dataframe
X = data[['portsrc', 'portdst', 'rule', '9']]
print(X.head())

print('\n')
y = data['action']
print(y.value_counts(normalize=True))

   portsrc portdst   rule    9
0  57630.0   443.0    1.0  6.0
1  63808.0   443.0    1.0  6.0
2  43123.0  2376.0  999.0  6.0
3   8088.0    70.0  999.0  6.0
4   6356.0   443.0    1.0  6.0


PERMIT    0.764711
DENY      0.235289
Name: action, dtype: float64


In [16]:
X.isna().sum()

portsrc    0
portdst    0
rule       0
9          0
dtype: int64

In [17]:
# Train test split
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [76]:
X_train.shape, X_test.shape

((14139032, 4), (3534758, 4))

In [18]:
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

(PERMIT    0.764597
 DENY      0.235403
 Name: action, dtype: float64,
 PERMIT    0.765165
 DENY      0.234835
 Name: action, dtype: float64)

In [31]:
from sklearn.ensemble import RandomForestClassifier

rf0 = RandomForestClassifier(n_estimators=100, random_state=0)

In [78]:
rf0.fit(X_train, y_train)

In [ ]:
# Random Forest Classifier with GridSearchCV 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a based model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                            cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)
